In [1]:
import datetime
import itertools
import numpy as np
from qiskit import Aer, IBMQ
from qiskit.providers.ibmq import IBMQBackend, IBMQJob
from mzm_generation import (
    KitaevHamiltonianTask,
    MeasurementErrorCalibrationTask,
    run_kitaev_hamiltonian_task,
    run_measurement_error_calibration_task,
    run_measurement_error_correction,
    save
)

In [2]:
# get backend
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-internal', group='deployed', project='default')
hardware_backend = provider.get_backend('ibmq_manila')
simulator_backend = Aer.get_backend("statevector_simulator")

In [3]:
# set parameters

# number of modes
n_modes = 3

# x values
num = 20
x_min = 1e-8
x_max = 3.1
x_values = np.linspace(x_min, x_max, num=num)

# backend
backend = simulator_backend

# qubits
# TODO choose qubits that minimize error rate
qubits = [0, 1, 2]

# number of shots
shots = 10000

# number of shots for measurement error calibration
measurement_error_calibration_shots = 50000

# experiment id used for saving data
experiment_id = f'{datetime.datetime.now().isoformat()}_{backend.name()}'

In [4]:
# generate tasks
tasks = []
for chemical_potential in x_values:
    for n_particles in range(n_modes + 1):
        for occupied_orbitals in itertools.combinations(range(n_modes), n_particles):
            task = KitaevHamiltonianTask(
                experiment_id=experiment_id,
                n_modes=n_modes,
                tunneling=-1.0,
                superconducting=1.0,
                chemical_potential=chemical_potential,
                occupied_orbitals=occupied_orbitals,
                shots=shots)
            tasks.append(task)

In [5]:
# run measurement error calibration
if isinstance(backend, IBMQBackend):
    measurement_error_calibration_task = MeasurementErrorCalibrationTask(experiment_id, shots=measurement_error_calibration_shots)
    run_measurement_error_calibration_task(measurement_error_calibration_task, backend, qubits)

In [6]:
# schedule jobs
# TODO save job information so they can be retrieved from provider later
jobs = []
for task in tasks:
    job = run_kitaev_hamiltonian_task(task, backend, qubits)
    jobs.append((task, job))

In [ ]:
# save results
for task, job in jobs:
    measurements = dict(zip(task.pauli_strings(), job.result().get_counts()))
    data = {
        'measurements': measurements
    }
    save(task, data)

In [5]:
# run measurement error mitigation and save results
for task in tasks:
    run_measurement_error_correction(measurement_error_calibration_task, task, qubits)